In [1]:
import torch
from torch.nn.utils import clip_grad_value_
import torch.optim as optim
import numpy as np
import pandas as pd
import pickle
import random
import re
from transformers import AutoTokenizer
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
MAX_LEN = 1200

In [2]:
version = "1"
MODEL_PATH = f"../Model/PRM_LORA{version}_merged_code_policy_01"
next_version = str(int(version) + 1)

#### Data: {1}

In [3]:
#TODO: clean_text
weight = 0.5
SFT_weight = 0.25

In [16]:
# separate out question and solution and only train on solution
patterns = ["``` and should only print the final answer.",\
            "print the final result.\nApproach:",\
            "print the final output, as an integer not other python object such as list or tuple."]
import re
clean_text = lambda x:re.sub(r"(<math>|<\/math>|<cmath>|<\/cmath>|\\begin\{align\*\}|\\end\{align\*\})", "", x)
def search_patterns(text, patterns):
    for pattern in patterns:
        # Compile the pattern
        regex = re.compile(pattern)
        # Find all matches of the pattern in the text
        matches = list(regex.finditer(text))
        # If there is one match, get the end position
        if matches:
            return matches[0].end()
    
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-math-7b-rl")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# # {1}
# import pickle
# with open(f"../llmOutputs/PRM/completed_paths_y_code{version}.pickle", "rb") as f:
#     completed_paths_y = pickle.load(f)

# texts = []
# for y,score,text,code,prob_i,exit_i in completed_paths_y:
#     if y == 1:
#         texts.append(text.replace("<｜begin▁of▁sentence｜>User: ",""))

# input_ids = []
# lengths = []
# for text in texts:
#     idx = search_patterns(text,patterns)
#     question = tokenizer.encode(text[:idx],add_special_tokens=True)
#     answer = tokenizer.encode(text[idx:],add_special_tokens=False)
#     lengths.append(len(question))
#     input_ids.append(question+answer)

In [8]:
# # {0,1}
# def from_gen(texts,ys,lengths):
#     data = list(zip(texts,ys,lengths))
#     random.shuffle(data)
#     for text,y,l in data:
#         text = torch.tensor(text[:MAX_LEN],device='cuda')[None]
#         yield text,y,l

# training
# for i,(text,y,l) in enumerate(from_gen(input_ids,ys,lengths)):
# loss = loss_fn(outs[0,l:-1],text[0,l+1:]) * y # (l,C), (l,)


# {1}
def from_gen(texts,lengths):
    data = list(zip(texts,lengths))
    random.shuffle(data)
    for text,l in data:
        text = torch.tensor(text[:MAX_LEN],device='cuda')[None]
        yield text,l

In [9]:
# {0,1}
with open(f"../llmOutputs/PRM/completed_paths_y_code{version}.pickle", "rb") as f:
    completed_paths_y = pickle.load(f)
data = []
for y,score,text,code,prob_i,exit_i in completed_paths_y:
    data.append([clean_text(text.replace("<｜begin▁of▁sentence｜>User: ","")),y])
texts,ys = zip(*data)

ys = np.array(ys)
ys = (ys-ys.mean())/ys.std()

input_ids = []
lengths = []
for text in texts:
    idx = search_patterns(text,patterns)
    question = tokenizer.encode(text[:idx],add_special_tokens=True)
    answer = tokenizer.encode(text[idx:],add_special_tokens=False)
    lengths.append(len(question))
    input_ids.append(question+answer)

In [10]:
# Pi
with open(f"../llmOutputs/PRM/data_pi1_code{version}.pickle", "rb") as f:
    data_pi = pickle.load(f)
texts2,ys2,lengths_raw = zip(*data_pi)
ys2 = np.array(ys2)
ys2 = ys2/ys2.std() * weight

# combined
ys = ys.tolist() + ys2.tolist()
for text,idx in zip(texts2,lengths_raw):
    question = tokenizer.encode(text[:idx].replace("<｜begin▁of▁sentence｜>User: ",""),add_special_tokens=True)
    answer = tokenizer.encode(text[idx:],add_special_tokens=False)
    lengths.append(len(question))
    input_ids.append(question+answer)

In [11]:
# # SFT
# def gen_prompt_codeIn1(problem):
#     return f"""Problem: {problem}\n
# To accomplish this, first determine a python-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and your final answer should be integer, not expression, list, tuple or dictionary!
# Write the entire script covering all the steps (use comments and document it well) and print the final result.
# Approach:"""

# def gen_prompt_codeIn2(problem):
#     return f"""Problem: {problem}\n
# You are an expert at solving math problem. Analyze this problem and think step by step to develop a python solution. Your solution should include reasoning steps in Python comments, explaining your thought process and the mathematical principles you applied. print the final output, as an integer not other python object such as list or tuple."""

# def add_prompt(problem):
#     if np.random.rand()<0.5:
#         return gen_prompt_codeIn1(problem)
#     else:
#         return gen_prompt_codeIn2(problem)
    
# sft = pd.read_csv("../Data/MATH/math.csv")
# sft = sft.loc[sft.boxed_number == sft.parsed] 
# sft['prob_wPrompt'] = sft.problem.apply(add_prompt)
# for q,a in zip(sft.prob_wPrompt.tolist(),sft.code_solution.tolist()):
#     question = tokenizer.encode(q,add_special_tokens=True)
#     answer = tokenizer.encode(a,add_special_tokens=False)
#     lengths.append(len(question))
#     input_ids.append(question+answer)

# ys = ys + [SFT_weight] * sft.shape[0]

In [ ]:
epochs = 1
accumulation_steps = 64
verbose = 1024
lr = 2e-5
clip = 2e-3
from transformers import AutoModelForCausalLM,BitsAndBytesConfig
import torch
from peft import (
    get_peft_model,
    PeftType,
    LoraConfig)
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH,\
                                            device_map="auto",
                                            torch_dtype="auto",
                                            quantization_config=quantization_config,
                                            trust_remote_code=True,
                                            attn_implementation="flash_attention_2"
                                            )
model.gradient_checkpointing_enable()
peft_config = LoraConfig(r=8, # low rank 
                         lora_alpha = 16, # see below 
                         lora_dropout = 0.1, 
                         bias="none",#'none', 'all' or 'lora_only' 
                         target_modules = [ "q_proj", 
                                            "k_proj", 
                                            "v_proj", 
                                            "o_proj", 
                                            "gate_proj", 
                                            "up_proj", 
                                            "down_proj" 
                                        ],
                        #  use_dora=True,
                        )
model = get_peft_model(model, peft_config)
model.gradient_checkpointing_enable()
# model.config.pad_token_id = tokenizer.pad_token_id
model.print_trainable_parameters()
trainable_params = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params,lr = lr)

In [17]:
def empty_cache():
    gc.collect()
    torch.cuda.empty_cache()

import math
import gc
loss_fn = torch.nn.CrossEntropyLoss()
train_loss = 0
count_loss = 0

for epoch in range(epochs):
    for i,(text,l) in enumerate(from_gen(input_ids,lengths)):
        if i > 0:
            del outs,loss
            empty_cache()
        outs = model(text).logits # 1,l,C
        loss = loss_fn(outs[0,l:-1],text[0,l+1:]) # (l,C), (l,)
        if math.isinf(loss.item()) or math.isnan(loss.item()): continue
        loss.backward()
        train_loss += loss.item()
        count_loss += 1
            
        if (i + 1) % accumulation_steps == 0:
            # clip_grad_value_(trainable_params,clip)
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()

        if (i + 1) % verbose == 0:
            print(f"iter: {i}, \n train loss: {train_loss/count_loss}")
            train_loss = 0
            count_loss = 0

iter: 1023, 
 train loss: 0.3026118021621187
iter: 2047, 
 train loss: 0.28984769291355406
iter: 3071, 
 train loss: 0.28957493073585283


In [18]:
next_version = str(int(version) + 1)
peft_model_id = f"../Model/PRM_LORA{next_version}_code_policy_01"
# !mkdir peft_model_id
model.save_pretrained(peft_model_id)

/home/zhenlan/anaconda3/envs/torch/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ../Model/PRM_LORA5_merged_code_policy_01 - will assume that the vocabulary was not modified.
  warnings.warn(


In [19]:
del model,texts,outs
gc.collect()
torch.cuda.empty_cache()
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH,\
                                    device_map="auto",
                                    torch_dtype="auto",
                                    attn_implementation="flash_attention_2"
                                    )
from peft import PeftModel
peft_model_id = f"../Model/PRM_LORA{next_version}_code_policy_01"
base_model = PeftModel.from_pretrained(model, peft_model_id)
base_model2 = base_model.merge_and_unload()
base_model2.save_pretrained(f"../Model/PRM_LORA{next_version}_merged_code_policy_01")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-math-7b-rl")
tokenizer.save_pretrained(f"../Model/PRM_LORA{next_version}_merged_code_policy_01")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('../Model/PRM_LORA7_merged_code_policy_01/tokenizer_config.json',
 '../Model/PRM_LORA7_merged_code_policy_01/special_tokens_map.json',
 '../Model/PRM_LORA7_merged_code_policy_01/tokenizer.json')